In [108]:
import os
import numpy as np
import scipy
from scipy.io import wavfile
import scipy.fftpack as fft
from scipy.signal import get_window
import matplotlib.pyplot as plt
import math
import librosa

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
import warnings
from sklearn.exceptions import DataConversionWarning

# kNN classifier code

In [ ]:
#Importing the required modules
import numpy as np
from scipy.stats import mode
 
#Euclidean Distance
def eucledian(p1,p2):
    dist = np.sqrt(np.sum((p1-p2)**2))
    return dist
 
#Function to calculate KNN
def predict(x_train, y , x_input, k):
    op_labels = []
    #Loop through the Datapoints to be classified
    for item in x_input: 
        #Array to store distances
        #point_dist = []
        point_dist=np.zeros(x_input.shape[0])
        #Loop through each training Data
        for j in range(len(x_train)): 
            distances = eucledian(np.array(x_train[j,:]) , item)#distance.euclidean()# 
            #Calculating the distance
            point_dist.append(distances) 
        point_dist = np.array(point_dist) 
        #Sorting the array while preserving the index, and keeping the first K datapoints
        dist = np.argsort(point_dist)[:k] 
        #Labels of the K datapoints from above
        labels = y[dist]
        #Majority voting using mode()
        lab = mode(labels) 
        lab = lab.mode[0]
        op_labels.append(lab)
 
    return op_labels

# Feature Extraction

## Vggish

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import librosa
vggmodel = hub.load('https://tfhub.dev/google/vggish/1')

def embedding_from_fn(fn):
    x, sr = librosa.load(fn,sr=None)
    x_16k = librosa.resample(x,sr,16000) #resample to 16KHz
    embedding = np.array(vggmodel(x_16k))
    return embedding

## Info csv load

In [2]:
relevant_genres = ['funk', 'hiphop', 'jazz', 'rock']

In [3]:
from scipy.io.wavfile import read as wavread
def ToolReadAudio(cAudioFilePath):    
    [samplerate, x] = wavread(cAudioFilePath)    
    if x.dtype == 'float32':        
        audio = x    
    else:        
        # change range to [-1,1)        
        if x.dtype == 'uint8':            
            nbits = 8        
        elif x.dtype == 'int16':            
            nbits = 16        
        elif x.dtype == 'int32':            
            nbits = 32        
        audio = x / float(2**(nbits - 1))    
        # special case of unsigned format    
    if x.dtype == 'uint8':        
        audio = audio - 1.    
    return (samplerate, audio)

In [19]:
import pandas as pd
info = pd.read_csv('./groove/info.csv')
#df=info[info['style'].isin(relevant_genres)]

#df.pivot(index='style', columns='beat_type', values='audio_filename')
df = pd.pivot_table(info, values='audio_filename', index=['style'],
                    columns=['beat_type'], aggfunc='count')
#df[df['beat'] >= 20]

info = info[info['style'].isin(relevant_genres)]
info = info[info['beat_type'] == "beat"]
info = info[info['audio_filename'].notna()].reset_index()
#locs = info['audio_filename'].to_numpy()


## Loading and Normalizing (and converting to Mono) Audio

In [49]:
info['audio_filename'][0]

'drummer1/session1/1_funk_80_beat_4-4.wav'

In [51]:
sr,audio = ToolReadAudio('./groove/'+'drummer1/session1/1_funk_80_beat_4-4.wav')
audio.shape[-1]

<ipython-input-3-70eaf167f057>:3: WavFileWarning: Chunk (non-data) not understood, skipping it.
  [samplerate, x] = wavread(cAudioFilePath)


2

In [92]:
info['audio'] = ''
for index, row in info.iterrows():
    #if index % 10 == 0:
        #print(index)
    try:
        sr,audio = ToolReadAudio('./groove/'+row['audio_filename'])
        if audio.shape[-1] == 2:
            info['audio'][index] = np.mean(audio,axis=1)
        else:
            info['audio'][index] = audio
        #info['audio'][index] = audio
    except Exception as e: # Some files are unable to be read by scipy's wavfile read, so skipping them and removing the corresponding rows
        info.drop(index,inplace=True)
        #print(e)
        
        print("Error, dropping row")
        print(f"size of info is now: {len(info)}")
info.reset_index(drop=True, inplace=True)
info.drop('index',axis=1,inplace=True)

<ipython-input-3-70eaf167f057>:3: WavFileWarning: Chunk (non-data) not understood, skipping it.
  [samplerate, x] = wavread(cAudioFilePath)
<ipython-input-92-771846dd385f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info['audio'][index] = np.mean(audio,axis=1)
<ipython-input-92-771846dd385f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info['audio'][index] = audio


In [91]:
info.drop('index',axis=1)

,drummer,session,id,style,bpm,beat_type,time_signature,midi_filename,audio_filename,duration,split,audio,mfcc_mean,mfcc_std,delta_mfcc_mean,delta_mfcc_std
0,drummer1,drummer1/session1,drummer1/session1/1,funk,80,beat,4-4,drummer1/session1/1_funk_80_beat_4-4.mid,drummer1/session1/1_funk_80_beat_4-4.wav,86.704688,train,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-432.616355373271, 22.49809168201099, 5.73710...","[126.14683462412935, 45.27268757902408, 30.796...","[0.016361130318489497, 0.014591610062187352, 0...","[24.464579923157032, 7.073338940494646, 4.8531..."
1,drummer1,drummer1/session1,drummer1/session1/102,funk,95,beat,4-4,drummer1/session1/102_funk_95_beat_4-4.mid,drummer1/session1/102_funk_95_beat_4-4.wav,224.807558,train,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-314.38503713137817, 40.66948674007691, -1.25...","[123.31143373316254, 53.301447409125586, 28.24...","[0.005671845408651794, 0.005318945980958389, 0...","[20.46855138991447, 7.462321078037354, 4.13203..."
2,drummer1,drummer1/session1,drummer1/session1/2,funk,80,beat,4-4,drummer1/session1/2_funk_80_beat_4-4.mid,drummer1/session1/2_funk_80_beat_4-4.wav,86.914062,validation,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-429.3064726116158, 28.13942860697189, 0.4587...","[121.35928220208608, 45.27376976757957, 32.948...","[0.011955841985556505, 0.009725588110931185, 0...","[21.93519441217284, 6.320677023666356, 4.55739..."
3,drummer1,drummer1/session1,drummer1/session1/3,funk,80,beat,4-4,drummer1/session1/3_funk_80_beat_4-4.mid,drummer1/session1/3_funk_80_beat_4-4.wav,122.851562,train,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-392.3593751347707, 29.777852838672406, 5.179...","[120.29496972754002, 49.27809279615074, 27.931...","[0.01585573678719116, 0.010809005107325562, -0...","[21.7130871759941, 6.501089514912341, 3.928144..."
4,drummer1,drummer1/session1,drummer1/session1/47,jazz,102,beat,4-4,drummer1/session1/47_jazz_102_beat_4-4.mid,drummer1/session1/47_jazz_102_beat_4-4.wav,183.415349,train,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-362.6509266419495, 68.25337942449839, -16.02...","[79.77445705108977, 42.751709372848346, 21.057...","[0.013213100454083937, 0.008112608671096536, -...","[13.423881940657083, 6.24281090680939, 2.69761..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,drummer9,drummer9/session1,drummer9/session1/5,rock,100,beat,4-4,drummer9/session1/5_rock_100_beat_4-4.mid,drummer9/session1/5_rock_100_beat_4-4.wav,71.780000,train,"[0.000244140625, 0.000396728515625, -0.0006408...",,,,
243,drummer9,drummer9/session1,drummer9/session1/6,rock,100,beat,4-4,drummer9/session1/6_rock_100_beat_4-4.mid,drummer9/session1/6_rock_100_beat_4-4.wav,4.748750,train,"[-0.16021728515625, -0.185333251953125, -0.183...",,,,
244,drummer9,drummer9/session1,drummer9/session1/7,rock,100,beat,4-4,drummer9/session1/7_rock_100_beat_4-4.mid,drummer9/session1/7_rock_100_beat_4-4.wav,23.938750,test,"[-0.001739501953125, 0.0018310546875, 0.004119...",,,,
245,drummer9,drummer9/session1,drummer9/session1/8,rock,100,beat,4-4,drummer9/session1/8_rock_100_beat_4-4.mid,drummer9/session1/8_rock_100_beat_4-4.wav,11.208750,train,"[0.033477783203125, -0.013763427734375, 0.0251...",,,,


In [114]:
def block_audio(x,blockSize,hopSize,fs):    
    # allocate memory    
    numBlocks = math.ceil(x.size / hopSize)    
    xb = np.zeros([numBlocks, blockSize])    
    # compute time stamps    
    t = (np.arange(0, numBlocks) * hopSize) / fs   
    x = np.concatenate((x, np.zeros(blockSize)),axis=0)    
    for n in range(0, numBlocks):        
        i_start = n * hopSize        
        i_stop = np.min([x.size - 1, i_start + blockSize - 1])        
        xb[n][np.arange(0,blockSize)] = x[np.arange(i_start, i_stop + 1)]    
    return (xb,t)

In [70]:
xb,t = block_audio(audio,2048,1024,44100)

In [33]:
def agg_mfccs(x):
    mfccs = librosa.feature.mfcc(x,n_mfcc = 13)
    mean = [np.mean(feature) for feature in mfccs]
    std = [np.std(feature) for feature in mfccs]
    mfcc_delta = librosa.feature.delta(mfccs)
    delta_mean=[np.mean(feature) for feature in mfcc_delta]
    delta_std=[np.std(feature) for feature in mfcc_delta]
    return mean,std,delta_mean,delta_std

In [158]:
def calc_stft(xb,fs=44100):
    stft = np.zeros((xb.shape[0],(int(xb[0].shape[0]/2)+1)))
    freqs = np.zeros((xb.shape[0],(int(xb[0].shape[0]/2)+1)))
    window = np.hanning(xb[0].shape[0])
    for i in range(xb.shape[0]):
        block= xb[i]
        # Apply Window to the block
        windowed_block = window * block 
        stft_blk = np.fft.fft(windowed_block)
        #stft_blk = np.fft.rfft(windowed_block)
        freq=np.fft.fftfreq(block.size,1/fs)
        freqs[i]=freq[:int(block.size/2)+1]
        stft_blk = np.abs(stft_blk)
        #stft_block = stft_blk[int((stft_blk.shape[0])/2):]
        stft_block = stft_blk[:int(((stft_blk.shape[0])/2)+1)]
        #stft_db = 10*np.log10(stft_block) # IS THIS CORRECT??
        stft[i]=stft_block#stft_db
    stft = np.array(stft)
    freqs=np.array(freqs)
    return stft,freqs

def extract_spectral_centroid(xb, fs):
    centroids = np.zeros(xb.shape[0])
    stft,freqs = calc_stft(xb,fs)
    #np.sum(magnitudes*freqs) / np.sum(magnitudes)
    for i in range(freqs.shape[0]):
        if np.all(stft[i] == 0):
            pass
        else:
            centroid = np.sum(stft[i]*freqs[i]) / np.sum(stft[i])
            centroids[i]=centroid
        #centroids.append(centroid)
    #centroids=np.array(centroids)
    return centroids

def extract_rms(xb):
    rms = np.zeros(xb.shape[0])
    for i in range(xb.shape[0]):
        if np.all(xb[i] == 0):
            r = 0.00001
        else:
            block = xb[i]
            r = np.sqrt(np.sum(block**2)/xb.shape[0])
            if r <= 0.00001: # Done to handle case when rms is 0 (for a block of all zeros
                r = 0.00001
            #rms.append(r)
        rms[i] = r
    #rms=np.array(rms)
    return 20*np.log10(rms)#rms,20*np.log10(rms) # What is reference value to convert to dB? Is it 1?

def extract_zerocrossingrate(xb):
    zcr= np.zeros(xb.shape[0])
    for i in range(xb.shape[0]):
        if np.all(xb[i] == 0):
            pass
        else:
            block = xb[i]
            zero_crossings = np.sum(np.abs(np.diff(np.sign(block)))) / block.shape[0] #np.nonzero(np.diff(block > 0))[0].size
            #zcr.append(zero_crossings)
            zcr[i] = zero_crossings 
    #zcr = np.array(zcr)
    return zcr

def extract_spectral_crest(xb):
    crest = np.zeros(xb.shape[0])
    stft,freqs = calc_stft(xb,44100)
    for i in range(stft.shape[0]):
        if np.all(stft[i] == 0):
            pass
        else:
            #crest.append((np.max(stft[i])/np.sum(stft[i])))
            crest[i] = np.max(stft[i])/np.sum(stft[i])
    #crest = np.array(crest)
    return crest

def extract_spectral_flux(xb):
    num_blocks=xb.shape[0] 
    blockSize = xb.shape[1]
    spectral_flux = np.zeros(num_blocks)
    stft,freqs = calc_stft(xb,44100)
    fft_len= stft.shape[1]
    n = 0
    k = 0
    for n in range(1,num_blocks):
        if np.all(stft[i] == 0):
            pass
        else:
            sum_flux = 0
            for k in range(fft_len):
                f = (abs(stft[n, k]) - abs(stft[n-1, k]))**2
                sum_flux += f
            flux = np.sqrt(sum_flux)/((blockSize/2)+1)
            spectral_flux[n] = flux #first flux value will be 0
    return spectral_flux


def extract_manual_features(x,fs):

    xb,ts = block_audio(x,2048,1024,fs)
    centroid = extract_spectral_centroid(xb,fs)
    rms = extract_rms(xb)
    zcr = extract_zerocrossingrate(xb)
    crest = extract_spectral_crest(xb)
    flux = extract_spectral_flux(xb)

    centroid_mean = np.mean(centroid)
    centroid_std = np.std(centroid)

    rms_mean = np.mean(rms)
    rms_std = np.std(rms)

    zcr_mean = np.mean(zcr)
    zcr_std = np.std(zcr)

    crest_mean = np.mean(crest)
    crest_std = np.std(crest)

    flux_mean = np.mean(flux)
    flux_std = np.std(flux)

    return centroid_mean, centroid_std, rms_mean, rms_std, zcr_mean, zcr_std, crest_mean, crest_std, flux_mean,flux_std




In [160]:
centroid_mean, centroid_std, rms_mean, rms_std, zcr_mean, zcr_std, crest_mean, crest_std, flux_mean,flux_std = extract_manual_features(df['audio'][0],44100)

<ipython-input-158-399a32098113>:49: RuntimeWarning: divide by zero encountered in log10
  return 20*np.log10(rms)#rms,20*np.log10(rms) # What is reference value to convert to dB? Is it 1?
/usr/local/lib/python3.9/site-packages/numpy/core/_methods.py:202: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


In [93]:
df=info
df['mfcc_mean'] = ''
df['mfcc_std'] = ''

df['delta_mfcc_mean'] = ''
df['delta_mfcc_std'] = ''

df['centroid_mean'] = ''
df['centroid_std'] = ''

df['rms_mean'] = ''
df['rms_std'] = ''

df['zcr_mean'] = ''
df['zcr_std'] = ''

df['crest_mean'] = ''
df['crest_std'] = ''

df['flux_mean'] = ''
df['flux_std'] = ''

for i in range(len(df)):
    #print(df['audio'][i].shape)
    mean,std,delta_mean,delta_std = agg_mfccs(df['audio'][i])
    # Extract 13 MFCCs - get mean and std deviation for each (26 features) + Delta MFCCs (26 features) = total 52 Features
    df['mfcc_mean'][i] = mean
    df['mfcc_std'][i] = std
    df['delta_mfcc_mean'][i] = delta_mean
    df['delta_mfcc_std'][i] = delta_std

    centroid_mean, centroid_std, rms_mean, rms_std, zcr_mean, zcr_std, crest_mean, crest_std, flux_mean,flux_std = extract_manual_features()


df[['mfcc1_mean','mfcc2_mean','mfcc3_mean','mfcc4_mean','mfcc5_mean','mfcc6_mean','mfcc7_mean','mfcc8_mean','mfcc9_mean','mfcc10_mean','mfcc11_mean','mfcc12_mean','mfcc13_mean']]=pd.DataFrame(df.mfcc_mean.tolist(), index= df.index)
df[['mfcc1_std','mfcc2_std','mfcc3_std','mfcc4_std','mfcc5_std','mfcc6_std','mfcc7_std','mfcc8_std','mfcc9_std','mfcc10_std','mfcc11_std','mfcc12_std','mfcc13_std']]=pd.DataFrame(df.mfcc_std.tolist(), index= df.index)

df[['delta_mfcc1_mean','delta_mfcc2_mean','delta_mfcc3_mean','delta_mfcc4_mean','delta_mfcc5_mean','delta_mfcc6_mean','delta_mfcc7_mean','delta_mfcc8_mean','delta_mfcc9_mean','delta_mfcc10_mean','delta_mfcc11_mean','delta_mfcc12_mean','delta_mfcc13_mean']]=pd.DataFrame(df.delta_mfcc_mean.tolist(), index= df.index)
df[['delta_mfcc1_std','delta_mfcc2_std','delta_mfcc3_std','delta_mfcc4_std','delta_mfcc5_std','delta_mfcc6_std','delta_mfcc7_std','delta_mfcc8_std','delta_mfcc9_std','delta_mfcc10_std','delta_mfcc11_std','delta_mfcc12_std','delta_mfcc13_std']]=pd.DataFrame(df.delta_mfcc_std.tolist(), index= df.index)


<ipython-input-93-0d9129188d51>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mfcc_mean'][i] = mean
<ipython-input-93-0d9129188d51>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mfcc_std'][i] = std
<ipython-input-93-0d9129188d51>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['delta_mfcc_mean'][i] = delta_mean
<ipython-input-93-0d9129188d51>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

In [115]:
df

,drummer,session,id,style,bpm,beat_type,time_signature,midi_filename,audio_filename,duration,...,delta_mfcc4_std,delta_mfcc5_std,delta_mfcc6_std,delta_mfcc7_std,delta_mfcc8_std,delta_mfcc9_std,delta_mfcc10_std,delta_mfcc11_std,delta_mfcc12_std,delta_mfcc13_std
0,drummer1,drummer1/session1,drummer1/session1/1,funk,80,beat,4-4,drummer1/session1/1_funk_80_beat_4-4.mid,drummer1/session1/1_funk_80_beat_4-4.wav,86.704688,...,1.864771,1.797111,1.362878,1.308343,1.334889,1.358760,1.366241,1.161793,0.966226,0.962835
1,drummer1,drummer1/session1,drummer1/session1/102,funk,95,beat,4-4,drummer1/session1/102_funk_95_beat_4-4.mid,drummer1/session1/102_funk_95_beat_4-4.wav,224.807558,...,1.884415,1.806498,1.330826,1.380146,1.379080,1.371703,1.304854,1.059812,1.017045,1.017376
2,drummer1,drummer1/session1,drummer1/session1/2,funk,80,beat,4-4,drummer1/session1/2_funk_80_beat_4-4.mid,drummer1/session1/2_funk_80_beat_4-4.wav,86.914062,...,1.955249,1.805681,1.397806,1.290850,1.198365,1.193234,1.258876,1.048738,0.897463,0.908775
3,drummer1,drummer1/session1,drummer1/session1/3,funk,80,beat,4-4,drummer1/session1/3_funk_80_beat_4-4.mid,drummer1/session1/3_funk_80_beat_4-4.wav,122.851562,...,2.013634,2.025345,1.354750,1.314860,1.198474,1.287613,1.208663,0.943772,0.900370,0.973241
4,drummer1,drummer1/session1,drummer1/session1/47,jazz,102,beat,4-4,drummer1/session1/47_jazz_102_beat_4-4.mid,drummer1/session1/47_jazz_102_beat_4-4.wav,183.415349,...,2.093440,1.422776,1.555978,1.574779,1.875317,1.150055,1.060434,0.949308,1.027447,0.852236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,drummer9,drummer9/session1,drummer9/session1/5,rock,100,beat,4-4,drummer9/session1/5_rock_100_beat_4-4.mid,drummer9/session1/5_rock_100_beat_4-4.wav,71.780000,...,2.563081,1.761732,1.410171,1.229826,1.575077,1.271688,1.170351,1.161183,1.136704,1.012200
243,drummer9,drummer9/session1,drummer9/session1/6,rock,100,beat,4-4,drummer9/session1/6_rock_100_beat_4-4.mid,drummer9/session1/6_rock_100_beat_4-4.wav,4.748750,...,2.197457,1.781863,1.580991,1.174969,1.842691,1.151712,1.077896,1.273932,0.874763,0.882934
244,drummer9,drummer9/session1,drummer9/session1/7,rock,100,beat,4-4,drummer9/session1/7_rock_100_beat_4-4.mid,drummer9/session1/7_rock_100_beat_4-4.wav,23.938750,...,1.892165,1.586016,1.322114,1.297345,1.174508,1.437726,1.192762,1.090079,1.132610,1.100330
245,drummer9,drummer9/session1,drummer9/session1/8,rock,100,beat,4-4,drummer9/session1/8_rock_100_beat_4-4.mid,drummer9/session1/8_rock_100_beat_4-4.wav,11.208750,...,2.157950,1.757571,1.478560,1.397111,1.390001,1.556802,1.335897,1.109056,1.246564,1.127368


# Classify

## Feature Normalization and train-test split

In [96]:
X = df[['mfcc1_mean', 'mfcc2_mean', 'mfcc3_mean',
       'mfcc4_mean', 'mfcc5_mean', 'mfcc6_mean', 'mfcc7_mean', 'mfcc8_mean',
       'mfcc9_mean', 'mfcc10_mean', 'mfcc11_mean', 'mfcc12_mean',
       'mfcc13_mean', 'mfcc1_std', 'mfcc2_std', 'mfcc3_std', 'mfcc4_std',
       'mfcc5_std', 'mfcc6_std', 'mfcc7_std', 'mfcc8_std', 'mfcc9_std',
       'mfcc10_std', 'mfcc11_std', 'mfcc12_std', 'mfcc13_std',
       'delta_mfcc1_mean', 'delta_mfcc2_mean', 'delta_mfcc3_mean',
       'delta_mfcc4_mean', 'delta_mfcc5_mean', 'delta_mfcc6_mean',
       'delta_mfcc7_mean', 'delta_mfcc8_mean', 'delta_mfcc9_mean',
       'delta_mfcc10_mean', 'delta_mfcc11_mean', 'delta_mfcc12_mean',
       'delta_mfcc13_mean', 'delta_mfcc1_std', 'delta_mfcc2_std',
       'delta_mfcc3_std', 'delta_mfcc4_std', 'delta_mfcc5_std',
       'delta_mfcc6_std', 'delta_mfcc7_std', 'delta_mfcc8_std',
       'delta_mfcc9_std', 'delta_mfcc10_std', 'delta_mfcc11_std',
       'delta_mfcc12_std', 'delta_mfcc13_std']]
y=df[['style']]

for col in X.columns:
    #df[col] = df[col]/max(np.abs(df[col]))
    mean = np.mean(X[col])
    std = np.std(X[col])
    X[col] = (X[col] - mean)/std # z-score normalization

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [124]:
X_train.T.shape

(52, 165)

In [112]:
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings('ignore', category=UndefinedMetricWarning) 
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

## Classification

In [126]:
print("-------------------------------------------------------------------------------")
print("            KNN with k=4              ")
print("-------------------------------------------------------------------------------")
k=4
KNN_model = KNeighborsClassifier(n_neighbors=k)
KNN_model.fit(X_train, y_train)
KNN_prediction = KNN_model.predict(X_test)
score=precision_score(y_test, KNN_prediction, average='macro')
accuracy = accuracy_score(y_test, KNN_prediction)
cm = confusion_matrix(y_test, KNN_prediction) # clean, highfry, layered, lowfry, midfry, no vocals
print(f"Confusion Matrix:\n {cm}")
print(f"Accuracy score: {accuracy}")
print(f"Macro accuracy score: {score}")

print("-------------------------------------------------------------------------------")
print("            SVM with C=1, kernel = rbf, degree = 3              ")
print("-------------------------------------------------------------------------------")

SVM_model = make_pipeline(StandardScaler(), SVC(gamma='auto',C=1.0, kernel='rbf', degree=3,random_state=None))
SVM_model.fit(X_train, y_train)
SVM_prediction = SVM_model.predict(X_test)
score=precision_score(y_test, SVM_prediction, average='macro')
accuracy = accuracy_score(y_test, SVM_prediction)
cm = confusion_matrix(y_test, SVM_prediction) # clean, highfry, layered, lowfry, midfry, no vocals
print(f"Confusion Matrix:\n {cm}")
print(f"Accuracy score: {accuracy}")
print(f"Macro accuracy score: {score}")

print("-------------------------------------------------------------------------------")
print("            RF with n_estimators = 100, criterion=gini, max_depth=None, min_samples_split=2,min_samples_leaf=1,max_features='auto', max_leaf_nodes=None,class_weight='balanced'              ")
print("-------------------------------------------------------------------------------")

RF_model = RandomForestClassifier(n_estimators=100,criterion='gini',max_depth=None,\
    min_samples_split=2,min_samples_leaf=1,max_features='auto',max_leaf_nodes=None,class_weight='balanced')
RF_model.fit(X_train, y_train)
RF_prediction = RF_model.predict(X_test)
score=precision_score(y_test, RF_prediction, average='macro')
accuracy = accuracy_score(y_test, RF_prediction)
cm = confusion_matrix(y_test, RF_prediction) # clean, highfry, layered, lowfry, midfry, no vocals
print(f"Confusion Matrix:\n {cm}")
print(f"Accuracy score: {accuracy}")
print(f"Macro accuracy score: {score}")

-------------------------------------------------------------------------------
            KNN with k=4              
-------------------------------------------------------------------------------
Confusion Matrix:
 [[ 9  0  0  2]
 [ 0  7  0  2]
 [ 0  0  2  8]
 [ 4  8  0 40]]
Accuracy score: 0.7073170731707317
Macro accuracy score: 0.732051282051282
-------------------------------------------------------------------------------
            SVM with C=1, kernel = rbf, degree = 3              
-------------------------------------------------------------------------------
Confusion Matrix:
 [[ 0  0  0 11]
 [ 0  0  0  9]
 [ 0  0  1  9]
 [ 0  0  0 52]]
Accuracy score: 0.6463414634146342
Macro accuracy score: 0.4104938271604938
-------------------------------------------------------------------------------
            RF with n_estimators = 100, criterion=gini, max_depth=None, min_samples_split=2,min_samples_leaf=1,max_features='auto', max_leaf_nodes=None,class_weight='balanced'          

# Old Unused code:

In [44]:
# def freq_to_mel(freq):
#     return 2595.0 * np.log10(1.0 + freq / 700.0)

# def mel_to_freq(mels):
#     return 700.0 * (10.0**(mels / 2595.0) - 1.0)

# def get_filter_points(fmin, fmax, mel_filter_num, FFT_size, fs=44100):
#     fmin_mel = freq_to_mel(fmin)
#     fmax_mel = freq_to_mel(fmax)
#     mels = np.linspace(fmin_mel, fmax_mel, num=mel_filter_num+2)
#     freqs = mel_to_freq(mels)
    
#     return np.floor((FFT_size + 1) / fs * freqs).astype(int), freqs

# def get_filters(filter_points, FFT_size):
#     filters = np.zeros((len(filter_points)-2,int(FFT_size/2+1)))
    
#     for n in range(len(filter_points)-2):
#         filters[n, filter_points[n] : filter_points[n + 1]] = np.linspace(0, 1, filter_points[n + 1] - filter_points[n])
#         filters[n, filter_points[n + 1] : filter_points[n + 2]] = np.linspace(1, 0, filter_points[n + 2] - filter_points[n + 1])
    
#     return filters
    
# def dct(dct_filter_num, filter_len):
#     basis = np.empty((dct_filter_num,filter_len))
#     basis[0, :] = 1.0 / np.sqrt(filter_len)
    
#     samples = np.arange(1, 2 * filter_len, 2) * np.pi / (2.0 * filter_len)

#     for i in range(1, dct_filter_num):
#         basis[i, :] = np.cos(i * samples) * np.sqrt(2.0 / filter_len)
        
#     return basis

In [75]:
# n_mfccs=13

# freq_min = 0
# freq_high = fs / 2
# fft_size=xb[0].shape[0]
# dct_filter_num = 40
# dct_filters = dct(dct_filter_num, n_mfccs)
# window = get_window("hann",fft_size , fftbins=True)

# audio_winT = np.transpose(xb*window)

# audio_fft = np.empty((int(1 + fft_size // 2), audio_winT.shape[1]), dtype=np.complex64, order='F')

# for n in range(audio_fft.shape[1]):
#     audio_fft[:, n] = fft.fft(audio_winT[:, n], axis=0)[:audio_fft.shape[0]]

# audio_fft = np.transpose(audio_fft)
# audio_power = np.square(np.abs(audio_fft))
# filter_points, mel_freqs = get_filter_points(freq_min, freq_high, n_mfccs, fft_size, fs)
# filters = get_filters(filter_points, fft_size)

# enorm = 2.0 / (mel_freqs[2:n_mfccs+2] - mel_freqs[:n_mfccs])
# filters *= enorm[:, np.newaxis]

# audio_filtered = np.dot(filters, np.transpose(audio_power))
# audio_log = 10.0 * np.log10(audio_filtered)

# dct_filters = dct(dct_filter_num, n_mfccs)

# cepstral_coefficents = scipy.fftpack.dct(audio_log)#np.dot(dct_filters, audio_log)


<ipython-input-75-42337085eaf8>:26: RuntimeWarning: divide by zero encountered in log10
  audio_log = 10.0 * np.log10(audio_filtered)


In [66]:
# def calc_mfccs(xb,fs=44100,n_mfccs=13):
#     freq_min = 0
#     freq_high = fs / 2
#     fft_size=xb[0].shape[0]
#     dct_filter_num = 40
#     dct_filters = dct(dct_filter_num, n_mfccs)
#     window = get_window("hann",fft_size , fftbins=True)
#     mfccs=[]
#     for block in xb:
#         # Apply Window to the block
#         windowed_block = window * block
#         stft_blk = np.fft.fft(windowed_block)[:windowed_block.shape[0]]
#         #stft_blk = np.fft.rfft(windowed_block)
#         stft_blk = np.square(np.abs(stft_blk)) #POWER SPECTRUM NOT MAGNITUDE
#         #stft_block = stft_blk[int((stft_blk.shape[0])/2):]
#         filter_points, mel_freqs = get_filter_points(freq_min, freq_high, n_mfccs, fft_size, fs)
#         filters = get_filters(filter_points, fft_size)
#         enorm = 2.0 / (mel_freqs[2:n_mfccs+2] - mel_freqs[:n_mfccs])
#         filters *= enorm[:, np.newaxis]
#         audio_filtered = np.dot(filters, np.transpose(stft_blk))
#         audio_log = 10.0 * np.log10(audio_filtered)
#         cepstral_coefficents = np.dot(dct_filters, audio_log)
#         mfccs.append(cepstral_coefficents)
#     return mfccs

In [69]:
window = get_window("hann",fft_size , fftbins=True)
windowed_block = window * block
stft_blk = np.fft.fft(windowed_block)
#stft_blk = np.fft.rfft(windowed_block)
stft_blk = np.square(np.abs(stft_blk)) #POWER SPECTRUM NOT MAGNITUDE
#stft_block = stft_blk[int((stft_blk.shape[0])/2):]
filter_points, mel_freqs = get_filter_points(freq_min, freq_high, n_mfccs, fft_size, fs)
filters = get_filters(filter_points, fft_size)
enorm = 2.0 / (mel_freqs[2:n_mfccs+2] - mel_freqs[:n_mfccs])
filters *= enorm[:, np.newaxis]
audio_filtered = np.dot(filters, np.transpose(stft_blk))
audio_log = 10.0 * np.log10(audio_filtered)
cepstral_coefficents = np.dot(dct_filters, audio_log)

ValueError: shapes (13,1025) and (2048,5294) not aligned: 1025 (dim 1) != 2048 (dim 0)

In [47]:

calc_mfccs(xb,44100,13)


ValueError: shapes (13,1025) and (2048,) not aligned: 1025 (dim 1) != 2048 (dim 0)

In [ ]:
def agg_mfccs(x):
    mfccs = librosa.feature.mfcc(x,n_mfcc = 13)
    mean = [np.mean(feature) for feature in mfccs]
    std = [np.std(feature) for feature in mfccs]
    mfcc_delta = librosa.feature.delta(mfccs)
    delta_mean=[np.mean(feature) for feature in mfcc_delta]
    delta_std=[np.std(feature) for feature in mfcc_delta]
    return mean,std,delta_mean,delta_std